<a href="https://colab.research.google.com/github/masterNLPIA2223/SeminarioComputacionProgramacion/blob/main/Python/Sesion4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 4: Trabajando con recursos online

En esta última sesión vamos a ver cómo trabajar con recursos online a través de Python, en concreto veremos qué son las [API Rest](https://www.redhat.com/es/topics/api/what-is-a-rest-api).

Veamos en primer lugar qué hace la librería requests. 

In [1]:
import requests

Con esta librería podemos obtener el contenido de una página web. 

In [3]:
respuesta = requests.get('https://es.wikipedia.org/wiki/Inteligencia_artificial')
respuesta

<Response [200]>

Ya vimos en la primera sesión cómo acceder al texto de la `Response` obtenida mediante el atributo `text`. 

In [4]:
respuesta.text

'<!DOCTYPE html>\n<html class="client-nojs" lang="es" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Inteligencia artificial - Wikipedia, la enciclopedia libre</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\\t.","\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"],"wgRequestId":"d6dcb121-08c2-4e0f-8f8f-6be9c4623712","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Inteligencia_artificial","wgTitle":"Inteligencia artificial","wgCurRevisionId":146051566,"wgRevisionId":146051566,"wgArticleId":1503,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:Artículos con pasajes que requieren referencias","Wikipedia:Artículos con

## APIs REST

Las APIs REST son recursos dinámicos que permiten acceder a funcionalidad que está disponible a través de un sitio web. Para usar una API REST es necesario conocer:
- La URL de la aplicación.
- Sus parámetros.

Vamos a comenzar con una API que sirve para reconocer entidades. Accede a la herramienta [LINDAT NameTag](http://lindat.mff.cuni.cz/services/nametag/) y prueba lo siguiente:
- Selecciona el modelo en castellano. 
- Pon algún texto (por ejemplo extraído de algún periódico).
- Explora los resultados. 

Vamos ahora a ver cómo utilizar su API con la siguiente frase:

In [5]:
frase = "Antonio Gaudí nació en Cataluña y construyó la Sagrada Familia"

Como hemos comentado previamente debemos encontrar la URL y los parámetros para usar esta API. Dicha información la puedes encontrar en la pestaña "REST API Documentation":
- url: "http://lindat.mff.cuni.cz/services/nametag/api/recognize"
- parámetros:
  - data : la frase que queremos analizar. 
  - model: el modelo que queremos utilizar. 
  - output: el tipo de salida. 


In [6]:
url = "http://lindat.mff.cuni.cz/services/nametag/api/recognize"

parameters = {
    "data": frase,
    "model": "spanish-conll-200831",
    "output": "vertical"
}

Ahora podemos realizar una solicitud con la librería `requests`.

In [13]:
res = requests.get(url, parameters)

res.text

'{\n "model": "spanish-conll-200831",\n "acknowledgements": [\n  "http://ufal.mff.cuni.cz/nametag/2#acknowledgements",\n  "http://ufal.mff.cuni.cz/nametag/2/models#spanish-conll_acknowledgements"\n ],\n "result": "1,2\\tPER\\tAntonio Gaudí\\n5\\tLOC\\tCataluña\\n9,10\\tMISC\\tSagrada Familia\\n"\n}\n'

La salida está almacenada en en formato [JSON](https://es.wikipedia.org/wiki/JSON), pero este formato aunque muy útil para la transferencia de información, resulta complejo de leer, por lo que hay que hacer un proceso de conversión mediante el método `json`.


In [15]:
resultado = res.json()
resultado

{'model': 'spanish-conll-200831',
 'acknowledgements': ['http://ufal.mff.cuni.cz/nametag/2#acknowledgements',
  'http://ufal.mff.cuni.cz/nametag/2/models#spanish-conll_acknowledgements'],
 'result': '1,2\tPER\tAntonio Gaudí\n5\tLOC\tCataluña\n9,10\tMISC\tSagrada Familia\n'}

Ahora ya tenemos un diccionario con el cúal podemos interactuar de la manera habitual. 

In [23]:
resultado["result"]

'1,2\tPER\tAntonio Gaudí\n5\tLOC\tCataluña\n9,10\tMISC\tSagrada Familia\n'

Como podemos ver este formato es poco legible, así que podemos mostrarlo mediante el método `print`.

In [18]:
print(resultado["result"])

1,2	PER	Antonio Gaudí
5	LOC	Cataluña
9,10	MISC	Sagrada Familia



Puede ser que este resultado lo queramos guardar en un DataFrame de pandas para guardarlo posteriormente en un fichero csv, para lo cual lo primero que tendremos que hacer es convertir el resultado en una lista de listas.

In [25]:
resultadoListaListas = []
# Separamos en primer lugar por los saltos de línea
for linea in resultado["result"].split('\n'):
  # Añadimos al resultado cada una de las listas, partiendo por el tabulador
  resultadoListaListas.append(linea.split('\t'))

resultadoListaListas

[['1,2', 'PER', 'Antonio Gaudí'],
 ['5', 'LOC', 'Cataluña'],
 ['9,10', 'MISC', 'Sagrada Familia'],
 ['']]

Vemos que hay una línea de más, por lo que vamos a quedarnos con todos los elementos de la lista salvo el último.

In [26]:
resultadoListaListas = resultadoListaListas[:-1]
resultadoListaListas

[['1,2', 'PER', 'Antonio Gaudí'],
 ['5', 'LOC', 'Cataluña'],
 ['9,10', 'MISC', 'Sagrada Familia']]

Ahora ya podemos construir nuestro DataFrame en pandas, y almacenarlo en un fichero. 

In [31]:
import pandas as pd

df = pd.DataFrame(resultadoListaListas,columns = ['pos','type','value'])
df.to_csv('entidades.csv',index=None)
df

,pos,type,value
0,"1,2",PER,Antonio Gaudí
1,5,LOC,Cataluña
2,"9,10",MISC,Sagrada Familia


Podemos ver que dicho fichero se ha guardado correctamente. 

In [32]:
pd.read_csv('entidades.csv')

,pos,type,value
0,"1,2",PER,Antonio Gaudí
1,5,LOC,Cataluña
2,"9,10",MISC,Sagrada Familia


Como siempre, es conveniente encapsular esta funcionalidad en una función para simplificar su uso.

In [33]:
def reconoce_entidades(frase,fichero_salida):
  url = "http://lindat.mff.cuni.cz/services/nametag/api/recognize"

  parameters = {
    "data": frase,
    "model": "spanish-conll-200831",
    "output": "vertical"
  }

  res = requests.get(url, parameters)

  resultado = res.json()
  resultadoListaListas = []
  for linea in resultado["result"].split('\n'):
    # Añadimos al resultado cada una de las listas, partiendo por el tabulador
    resultadoListaListas.append(linea.split('\t'))

  resultadoListaListas = resultadoListaListas[:-1]
  df = pd.DataFrame(resultadoListaListas,columns = ['pos','type','value'])
  df.to_csv(fichero_salida,index=None)

In [34]:
reconoce_entidades("Rafael Nadal Parera (Manacor, Mallorca, España, 3 de junio de 1986), más conocido como Rafa Nadal, es un tenista profesional español.","entidades.csv")

In [35]:
pd.read_csv('entidades.csv')

,pos,type,value
0,"1,2,3",PER,Rafael Nadal Parera
1,5,LOC,Manacor
2,7,LOC,Mallorca
3,9,LOC,España
4,"21,22",PER,Rafa Nadal
